<a href="https://colab.research.google.com/github/C4viar/PPD_MSLD_2019_2020_LL-AL-MT/blob/master/Draft_code/projet_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montage du drive

In [6]:
# Montage du drive

from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Contexte


Des modèles de langage du français ont tout récemment été publiés (CamemBERT, FlauBERT C’est positif car même des outils avancés comme Spacy sont notoirement encore peu adaptés à traiter certaines langues dont le français. Il reste à évaluer ce que ces modèles peuvent vraiment apporter notamment ne ce qui concerne la reconnaissances d’entités nommées.


# Travail a réaliser

Réaliser des expériences de reconnaissance d’entités nommées sur le jeu de donneés Europeana Newspapers French en utilisant tous les modèles disponibles pour cela dans la bibliothèque Simple Transformers. Les adresses à consulter sont :
* https://github.com/ThilinaRajapakse/simpletransformers
* http://api.bnf.fr/europeana-newspapers-french-ner-ground-truth

# Rendu

Un code clair et commenté dans un notebook.
Un rapport décrivant les méthodes/métriques d’évaluation (basées sur seqeval dont une description détaillée sera donnée) et présentant/comparant/discutant les résultats obtenus.


# Seqeval 

https://pypi.org/project/seqeval/

"seqeval is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on."

# Installation simple transformers

In [0]:
pip install simpletransformers

In [0]:
pip install wandb

# Installation APEX

In [8]:
! git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 6733 (delta 91), reused 77 (delta 49), pack-reused 6596
Receiving objects: 100% (6733/6733), 13.76 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (4480/4480), done.


In [9]:
cd apex

/content/apex


In [10]:
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-2fhz8rsb
Created temporary directory: /tmp/pip-req-tracker-7beq2rgj
Created requirements tracker '/tmp/pip-req-tracker-7beq2rgj'
Created temporary directory: /tmp/pip-install-ssszx1_h
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-e6qs5eo1
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-7beq2rgj'
    Running setup.py (path:/tmp/pip-req-build-e6qs5eo1/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-e6qs5eo1/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-e6qs5eo1/pip-egg-info/apex.egg-info/PKG-I

# NER 
https://simpletransformers.ai/docs/ner-specifics/

"The goal of Named Entity Recognition is to locate and classify named entities in a sequence. The named entities are pre-defined categories chosen according to the use case such as names of people, organizations, places, codes, time notations, monetary values, etc. Essentially, NER aims to assign a class to each token (usually a single word) in a sequence. Because of this, NER is also referred to as token classification."

**Fonctionnement avec Simple Transformer**

* Initialize a NERModel
* Train the model with train_model()
* Evaluate the model with eval_model()
* Make predictions on (unlabelled) data with predict()

**Modèles disponibles à fournir à model_type** : https://huggingface.co/models

* BERT :	*bert*
* CamemBERT :	*camembert*
* RoBERTa :	*roberta*
* DistilBERT :	*distilbert*
* ELECTRA	: *electra*
* XLM-RoBERTa	: *xlmroberta*

In [0]:
from simpletransformers.ner import NERModel

# to custom labels
custom_labels = ["O", "B-SPELL", "I-SPELL", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-PLACE", "I-PLACE"]


model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True}, labels=custom_labels)

In [11]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Creating train_df  and eval_df for demonstration
train_data = [
    [0, 'Simple', 'B-MISC'], [0, 'Transformers', 'I-MISC'], [0, 'started', 'O'], [1, 'with', 'O'], [0, 'text', 'O'], [0, 'classification', 'B-MISC'],
    [1, 'Simple', 'B-MISC'], [1, 'Transformers', 'I-MISC'], [1, 'can', 'O'], [1, 'now', 'O'], [1, 'perform', 'O'], [1, 'NER', 'B-MISC']
]
train_df = pd.DataFrame(train_data, columns=['sentence_id', 'words', 'labels'])

eval_data = [
    [0, 'Simple', 'B-MISC'], [0, 'Transformers', 'I-MISC'], [0, 'was', 'O'], [1, 'built', 'O'], [1, 'for', 'O'], [0, 'text', 'O'], [0, 'classification', 'B-MISC'],
    [1, 'Simple', 'B-MISC'], [1, 'Transformers', 'I-MISC'], [1, 'then', 'O'], [1, 'expanded', 'O'], [1, 'to', 'O'], [1, 'perform', 'O'], [1, 'NER', 'B-MISC']
]
eval_df = pd.DataFrame(eval_data, columns=['sentence_id', 'words', 'labels'])

# Create a NERModel
model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True})

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

# Predictions on arbitary text strings
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

print(predictions)

INFO:simpletransformers.ner.ner_model: Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 2.324800Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 2.2720589637756348, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0}


INFO:simpletransformers.ner.ner_model: Converting to features started.



[[{'Some': 'I-MISC'}, {'arbitary': 'I-MISC'}, {'sentence': 'I-MISC'}]]


In [0]:
import pandas as pd 

In [21]:
cd drive/My Drive/PPD-NER

/content/drive/My Drive/PPD-NER


In [0]:
ls ./data

In [0]:
import os
files = os.listdir("./data")

In [0]:
l_bug = []
l_ok = []
df = pd.DataFrame()
from pandas.io.parsers import ParserError
for i in files :
  try:
    df = df.append(pd.read_csv("./data/"+i, sep = "\t",doublequote = False, header = None))
    l_ok.append(i)
  except ParserError as err :
    l_bug.append(i)

In [0]:
test = pd.read_csv("./data/xbh.M.tag", sep = "\t", header = None)

In [0]:
test.columns = ["words", "labels"]

In [0]:
tags = test["tag"].unique().tolist()

In [0]:
j = 0
test["sentence_id"] = ""
for i in range(test.shape[0]):
  if (test.loc[i, "words"] in ([".", "!", "?"])):
    test.loc[i, "sentence_id"] = j
    j = j+1
  else :
    test.loc[i, "sentence_id"] = j

In [0]:
test = test[["sentence_id", "words", "labels"]]

In [127]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Creating train_df  and eval_df for demonstration

train_df = test.iloc[0:800]

eval_df = test.iloc[800:]

# Create a NERModel
model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True}, labels=tags)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

# Predictions on arbitary text strings
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

print(predictions)

INFO:simpletransformers.ner.ner_model: Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.408997Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.403135Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.379773Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.407403

...

Running loss: 1.066118


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.9032625555992126, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0}
INFO:simpletransformers.ner.ner_model: Converting to features started.



[[{'Some': 'O'}, {'arbitary': 'O'}, {'sentence': 'O'}]]


In [128]:
predictions

[[{'Some': 'O'}, {'arbitary': 'O'}, {'sentence': 'O'}]]